### Student's t-test  

Assumptions:
- Normally distributed
- Homogeneous variance

Null hypothesis: The means of two samples are equal  

Unpaired

In [1]:
from scipy.stats import ttest_ind
import numpy as np

df1 = np.random.normal(20, 1, 23)
df2 = np.random.normal(20.8, 1, 23)

stat, p = ttest_ind(df1, df2)

print(f"T-Statistic: {stat:.2f}\np-value: {p:.3f}")

T-Statistic: -2.47
p-value: 0.017


Calculating effect size

In [3]:
pooled_std = np.sqrt((df1.std()**2 + df2.std()**2)/2)
cohens = (np.mean(df2) - np.mean(df1)) / pooled_std
print(f"Cohen's d: {cohens}")

Cohen's d: 0.7453992454457741


Minimum detectable effect size

In [7]:
from statsmodels.stats.power import TTestIndPower

solver = TTestIndPower()

power = 0.8
alpha = 0.05

min_effect_size = solver.solve_power(effect_size=None, power=power, alpha=alpha, nobs1=23, ratio=1)
print(f"Minimum effect size detectable at a = {alpha}, power = {power}, 23 observations: {min_effect_size:.4f}")

Minimum effect size detectable at a = 0.05, power = 0.8, 23 observations: 0.8447


Minimum sample size

In [14]:
min_samples = solver.solve_power(effect_size=cohens, nobs1=None, alpha=alpha, power=power, ratio=1)

print(f"Minimum number of observations per sample: {np.ceil(min_samples):.0f}")

Minimum number of observations per sample: 30


Redoing experiment with new samples, n = 30

In [16]:
df1 = np.random.normal(20, 1, 30)
df2 = np.random.normal(20.8, 1, 30)

stat, p = ttest_ind(df1, df2)
print(f"T-Statistic: {stat:.2f}\np-value: {p:.3f}")

pooled_std = np.sqrt((df1.std()**2 + df2.std()**2)/2)
cohens = (np.mean(df2) - np.mean(df1)) / pooled_std
print(f"Cohen's d: {cohens}")

min_effect_size = solver.solve_power(effect_size=None, power=power, alpha=alpha, nobs1=len(df1), ratio=1)
print(f"Minimum effect size detectable at a = {alpha}, power = {power}, {len(df1)} observations: {min_effect_size:.4f}")

T-Statistic: -3.72
p-value: 0.000
Cohen's d: 0.9780544130326311
Minimum effect size detectable at a = 0.05, power = 0.8, 30 observations: 0.7356


Paired

In [19]:
from scipy.stats import ttest_rel
from statsmodels.stats.power import TTestPower

solver = TTestPower()

# Draw initial sample of size 18
df1 = np.random.normal(20, 1, 18)
df2 = df1 + np.random.normal(0.6, 0.7, 18)

# decide type 1 and 2 error tolerances
power = 0.9
alpha = 0.05

# calculate difference of two initial samples, calculate minimum samples
pooled_std = np.sqrt((df1.std()**2 + df2.std()**2)/2)
cohens = (np.mean(df2) - np.mean(df1)) / pooled_std
min_samples = int(np.ceil(solver.solve_power(effect_size=cohens, nobs=None, alpha=alpha, power=power))) + 1

# draw number of samples required for the ttest
df1 = np.random.normal(20, 1, min_samples)
df2 = df1 + np.random.normal(0.6, 0.7, min_samples)

# calculate the new cohens
pooled_std = np.sqrt((df1.std()**2 + df2.std()**2)/2)
cohens = (np.mean(df2) - np.mean(df1)) / pooled_std

# do the test
stat, p = ttest_rel(df1, df2)
min_effect_size = solver.solve_power(effect_size=None, power=power, alpha=alpha, nobs=len(df1))
print(f"Minimum effect size detectable at a = {alpha}, power = {power}, {len(df1)} observations: {min_effect_size:.4f}")
print(f"Cohen's d: {cohens}")
print(f"T-Statistic: {stat:.2f}\np-value: {p:.3f}")

Minimum effect size detectable at a = 0.05, power = 0.9, 56 observations: 0.4409
Cohen's d: 0.5175056587849609
T-Statistic: -6.28
p-value: 0.000


One-sample

In [12]:
from scipy.stats import ttest_1samp
import warnings
warnings.filterwarnings("ignore")

stat, p = ttest_1samp(np.random.normal(3.1, 1, 8), 3)

print(f"T-Statistic: {stat:.2f}\np-value: {p:.3f}")

T-Statistic: -2.02
p-value: 0.083


### Analysis of Variance Test (ANOVA)  

Assumptions:
- Normally distributed
- Homogeneous variance

Null hypothesis: The means of samples are equal  

Unpaired

In [5]:
from scipy.stats import f_oneway

df1 = np.random.normal(20, 1, 23)
df2 = np.random.normal(20.2, 1, 23)
df3 = np.random.normal(19.8, 1, 23)
df4 = np.random.normal(20.3, 1, 23)

stat, p = f_oneway(df1, df2, df3, df4)

print(f"Statistic: {stat:.2f}\np-value: {p:.3f}")

Statistic: 0.93
p-value: 0.431


Paired

In [6]:
from statsmodels.stats.anova import AnovaRM
import pandas as pd

df1 = np.random.normal(20, 1, 23)
df2 = df1 + np.random.normal(0.6, 0.7, 23)
df3 = df1 + np.random.normal(0.1, 0.3, 23)

together = np.concatenate([df1, df2, df3])
d_from = [[i]*23 for i in range(3)]
d_from = d_from[0] + d_from[1] + d_from[2]
patient = list(range(23)) * 3

(df := pd.DataFrame({"drug":d_from, "patient":patient, "effect":together}))

,drug,patient,effect
0,0,0,20.274116
1,0,1,19.198682
2,0,2,20.689133
3,0,3,20.957441
4,0,4,21.999473
...,...,...,...
64,2,18,21.145065
65,2,19,19.682705
66,2,20,20.920504
67,2,21,21.420899


In [7]:
print(AnovaRM(df, "effect", "patient", within=["drug"]).fit())

              Anova
     F Value Num DF  Den DF Pr > F
----------------------------------
drug  7.0080 2.0000 44.0000 0.0023

